### Import Libraries and Query Aletheia


In [1]:
import ipywidgets as widgets
import xdcm_dash_functions as f
from matplotlib import rcParams

%load_ext autoreload
%autoreload 2
%matplotlib inline

from xdcm_dash_functions import *

"""
Colorization and visualization utilities.
"""

import numpy as np
import skimage.transform as skt

#from brtlearn.shared import config
PIX_DATA_CLASS_DIRT = 0
PIX_DATA_CLASS_WEED = 1
PIX_DATA_CLASS_CROP = 2

from datetime import datetime

# filter out images that are not NRG (e.g no RGB images), robots that are not DCMS and images with no crop label. 
start = datetime(2020, 3,7)

# filter images by robot_name, image type, crop, etc
filters = {"artifacts.kind": "nrg", "crop_name": {"$ne": None}, "robot_name": {"$in": ["DCM-MANATEE", 
                            "DCM-DOLPHIN", "DCM-WALRUS", "DCM-OTTER","DCM-PORPOISE", "DCM-SEAL"]} }

full_df, elapsed_time = f.get_shasta_data(filters=filters, start=start)
full_df['date_collected'] = pd.to_datetime(full_df['collected_on'].dt.date)
print(f"Queried {len(full_df)} images in {elapsed_time:.2f} s.")

Queried 185069 images in 201.25 s.


In [9]:
# Search by crop size

qdf = full_df.copy()
sizes = qdf.groupby('crop_damage').count()
sizes

TypeError: unhashable type: 'list'

In [3]:
print(full_df.columns)

Index(['_id', 'is_deleted', 'collected_on', 'gnss', 'annotations', 'artifacts',
       'has_human_annotation', 'uuid', 'angle_to_row', 'camera_exposure_algo',
       'camera_exposure_time', 'camera_focal_length', 'camera_fstop',
       'camera_gain', 'camera_manufacturer', 'camera_revision',
       'camera_sensor_name', 'camera_sensor_type', 'camera_serial',
       'cloud_cover', 'crop_damage', 'crop_health', 'crop_height', 'crop_name',
       'crop_residue', 'experiment_id', 'farm', 'furrow_depth_inches',
       'gps_course', 'gps_speed', 'group_id', 'grower', 'icp_version',
       'operating_field_name', 'operator_name', 'project_name',
       'raw_bit_depth', 'row_spacing', 'soil_color', 'tillage_practice',
       'type', 'weed_pressure', 'weeds', 'robot_name',
       'artifacts_content_hash', 'notes', 'camera_height', 'camera_ppi',
       'gps_quality', 'isp_version', 'latitude', 'longitude',
       'date_collected'],
      dtype='object')


### Search for Damaged Crops


In [ ]:
# Query Crop Damage 

#print(full_df.columns)

damage_type = 'CUPPING'

june = full_df[(full_df['date_collected']> '2020-04-30') & (full_df['crop_name'] == 'SOYBEANS')]
june['crop_damage_string'] = [','.join(map(str, l)) for l in june['crop_damage']]
damaged_fields = june[june['crop_damage_string'] == damage_type]
damaged_fields.groupby('operating_field_name').count()

june = full_df[(full_df['date_collected']> '2020-03-30') & (full_df['crop_name'] == 'SOYBEANS')]
june['crop_damage_string'] = [','.join(map(str, l)) for l in june['crop_damage']]

dam_soy = june.groupby('crop_damage_string').count()
print(dam_soy)

ljn = june[june['operating_field_name']=='lost john north']
print(ljn.head())

ljn.tail()
#df['Feature3'] = df['dic'].apply(lambda x: x.get('Feature3'))
pdc =ljn['gnss'].apply(pd.Series)
pdc